In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import seaborn as sns
import collections as cll
import itertools
import glob
import os
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras import regularizers
from keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, BatchNormalization, AveragePooling2D, LSTM
from keras.optimizers import SGD
from keras.optimizers import Adam, Nadam, Adadelta
from keras.models import load_model
from sklearn.metrics import classification_report,confusion_matrix
from keras.datasets import mnist
from keras.utils import Sequence
from keras.preprocessing.sequence import pad_sequences

class My_generator(Sequence):
    def __init__(self, x_set_dir, y_set, batch_size):
        self.x, self.y = x_set_dir, y_set
        self.batch_size = batch_size
        
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        # read your data here using the batch lists, batch_x and batch_y
        x = [np.load(filename) for filename in batch_x]
        y = [np.load(filename) for filename in batch_y]
    
        return np.array(x).reshape(self.batch_size,1,n_sample,n_feat), np.array(y) 

Using TensorFlow backend.


In [2]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    fig = plt.figure()
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.4f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Classe correta')
    plt.xlabel('Classe prevista')
    
    return fig

In [3]:
n_sample, n_feat = 6410, 8
n_classes = 2
input_shape = (1,n_sample,n_feat)

# B1

In [4]:
train_path = glob.glob(os.path.join('dados/cruzinhoB1/train/', '*.npy'))
test_path = glob.glob(os.path.join('dados/cruzinhoB1/test', '*.npy'))
y_train_dir = glob.glob(os.path.join('dados/cruzinhoB1/B1_y_train/', '*.npy'))
y_test_dir = glob.glob(os.path.join('dados/cruzinhoB1/B1_y_test/', '*.npy'))

In [5]:
print(len(train_path))
print(len(test_path))

8169
2043


In [6]:
batch_size_train = 3
batch_size_test = 3
my_training_batch_generator = My_generator(train_path, y_train_dir, batch_size_train)
my_validation_batch_generator = My_generator(test_path, y_test_dir, batch_size_test)

In [7]:
def cnn_model(input_dim, n_classes, init,drop):
    model = Sequential()
    model.add(Conv2D(256, (3,3), strides=1, kernel_initializer=init , padding='same', activation='relu', input_shape=input_dim))
    model.add(Conv2D(256, (3,3), strides=1, kernel_initializer=init , padding='same', activation='relu'))
    model.add(AveragePooling2D(pool_size=(1,9)))
    model.add(Dropout(drop))
    
    model.add(Conv2D(512, (3,3), strides=1, kernel_initializer=init , padding='same', activation='relu'))
    model.add(Conv2D(512, (3,3), strides=1, kernel_initializer=init , padding='same', activation='relu'))
    model.add(AveragePooling2D(pool_size=(1,6)))
    model.add(Dropout(drop))
    
    model.add(Conv2D(768, (3,3), strides=1, kernel_initializer=init , padding='same', activation='relu'))
    model.add(Conv2D(768, (3,3), strides=1, kernel_initializer=init , padding='same', activation='relu'))
    model.add(AveragePooling2D(pool_size=(1,4)))
    model.add(Dropout(drop))
    
    model.add(Conv2D(1024, (3,3), strides=1, kernel_initializer=init , padding='same', activation='relu'))
    model.add(Conv2D(1024, (3,3), strides=1, kernel_initializer=init , padding='same', activation='relu'))
    model.add(AveragePooling2D(pool_size=(1,3)))
    model.add(Dropout(drop))
    
    model.add(Flatten())
    model.add(Dense(4096, kernel_initializer=init , activation='relu'))
    model.add(Dropout(drop*6))

    model.add(Dense(768, kernel_initializer=init , activation='relu'))
    model.add(Dropout(drop*2))

    return model

In [8]:
init = keras.initializers.he_uniform(42)
model = cnn_model(input_shape,n_classes,init,drop=0.05)
model.add(Dense(n_classes,activation='softmax'))
#model.load_weights('PesosB1/cruzinho-CNN-vgg-new.71-1.02-0.88-4.18-0.64.hdf5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1, 6410, 256)      18688     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 6410, 256)      590080    
_________________________________________________________________
average_pooling2d_1 (Average (None, 1, 712, 256)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 712, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1, 712, 512)       1180160   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 712, 512)       2359808   
_________________________________________________________________
average_pooling2d_2 (Average (None, 1, 118, 512)       0         
__________

In [ ]:
sgd = SGD(lr=0.1, decay=0, momentum=0.9, nesterov=True)
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0, amsgrad=False)
nadam = Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.000001)
adadelta = Adadelta(lr = 1)
filepath = 'PesosB1/cruzinho-CNN-vgg-new2.{epoch:02d}-{loss:.2f}-{acc:.2f}-{val_loss:.2f}-{val_acc:.2f}.hdf5'
cb = keras.callbacks.ModelCheckpoint(filepath, monitor='acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_red = keras.callbacks.ReduceLROnPlateau(monitor='acc', patience=3, verbose=1, factor=0.5, min_lr=0.000001)

model.compile(optimizer = nadam, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit_generator(generator=my_training_batch_generator,
                                          epochs=75,
                                          verbose=1,
                                          use_multiprocessing=True,
                                          workers = 8,
                                          max_queue_size=84, 
                                          validation_data=my_validation_batch_generator, 
                                          validation_steps=(len(test_path) // batch_size_test),
                                          steps_per_epoch=(len(train_path) // batch_size_train),
                                          callbacks = [cb, lr_red],
                                          initial_epoch=0)

Epoch 1/75
2723/2723 [==============================] - 1005s 369ms/step - loss: 1.7475 - acc: 0.5713 - val_loss: 1.7204 - val_acc: 0.5825
Epoch 2/75
2723/2723 [==============================] - 898s 330ms/step - loss: 1.5369 - acc: 0.5794 - val_loss: 2.1115 - val_acc: 0.5825
Epoch 3/75
2723/2723 [==============================] - 898s 330ms/step - loss: 1.4245 - acc: 0.6277 - val_loss: 3.5687 - val_acc: 0.6148
Epoch 4/75
2723/2723 [==============================] - 896s 329ms/step - loss: 1.4351 - acc: 0.6426 - val_loss: 3.1286 - val_acc: 0.6383
Epoch 5/75
2723/2723 [==============================] - 922s 339ms/step - loss: 1.2581 - acc: 0.6605 - val_loss: 1.2905 - val_acc: 0.6461
Epoch 6/75
2723/2723 [==============================] - 959s 352ms/step - loss: 1.3467 - acc: 0.6576 - val_loss: 1.4216 - val_acc: 0.6422
Epoch 7/75
2280/2723 [========================>.....] - ETA: 2:22 - loss: 1.2183 - acc: 0.6654

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
y_test = []
y_train = []

for i in y_test_dir:
    tmp = np.load(i)
    y_test.append(tmp)

for i in y_train_dir:
    tmp = np.load(i)
    y_train.append(tmp)

In [ ]:
#Predict Test
classes = ['sem falha', 'com falha']
y_testing = np.argmax(y_test, axis =1)
pred = model.predict_generator(my_validation_batch_generator, workers=8, use_multiprocessing=True, verbose=1,max_queue_size=64, steps=(len(test_path) // batch_size_test))
pred = np.argmax(pred, axis = 1)
print(classification_report(y_testing,pred,digits=4))
cm = confusion_matrix(y_testing,pred)
cm_plot = plot_confusion_matrix(cm, classes, normalize=True, title='')
cm_plot.savefig('cruzin_b1_test.png',dpi = 'figure', bbox_inches='tight')

In [ ]:
#Predict Train
y_training = np.argmax(y_train, axis = 1)
pred = model.predict_generator(my_training_batch_generator, workers=8, use_multiprocessing=True, verbose=1,max_queue_size=64, steps=(len(train_path) // batch_size_train))
pred = np.argmax(pred, axis = 1)
print(classification_report(y_training,pred,digits=4))
cm = confusion_matrix(y_training,pred)
cm_plot = plot_confusion_matrix(cm, classes, normalize=True, title='')
cm_plot.savefig('cruzin_b1_train.png',dpi = 'figure', bbox_inches='tight')